In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import scipy
import datetime
import numpy as np
import tensorflow as tf
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import random
import sklearn.linear_model
import scipy.optimize
import sklearn.decomposition
import sklearn.manifold
import sklearn.model_selection

In [3]:
# Libraries for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.manifold

sns.set_context('poster')

In [13]:
path_to_simclr_hchs = os.path.dirname(os.path.dirname(os.getcwd()))
path_to_hchs_dataset = os.path.join(os.path.dirname(path_to_simclr_hchs), "Datasets", "hchs")
path_to_actigraphy_data = os.path.join(path_to_hchs_dataset, "actigraphy")
path_to_baseline_dataset = os.path.join(path_to_hchs_dataset, "datasets", "hchs-sol-baseline-dataset-0.5.0.csv")
path_to_sueno_dataset = os.path.join(path_to_hchs_dataset, "datasets", "hchs-sol-sueno-ancillary-dataset-0.5.0.csv")
dataset_save_path = os.path.join(path_to_simclr_hchs, "PickledData", "hchs")
user_dataset_path = os.path.join(dataset_save_path, "user_dataset_resized.pickle")
path_to_baseline_sueno_merge_no_na_for_supervised = os.path.join(dataset_save_path, "baseline_sueno_merge_no_na_for_supervised.pickle")


In [14]:
user_dataset_path

'c:\\Users\\Kevalee Shah\\Documents\\Cambridge\\Part II\\Project\\SimCLR HCHS\\PickledData\\hchs\\user_dataset_resized.pickle'

In [15]:
disease_labels = {'diabetes': {1: 'Non-diabetic', 2: 'Pre-diabetic', 3: 'Diabetic'}, 'sleep_apnea': {0: 'No', 1: 'Yes'}, 'hypertension': {0: 'No', 1: 'Yes'}, 'metabolic_syndrome': {0: 'No', 1: 'Yes'}, 'insomnia': {1: 'No clinically significant insomnia', 2: 'Subthreshold insomnia', 3: 'Clinical insomnia'}}


In [20]:
with open(user_dataset_path, 'rb') as f:
    user_dataset = pickle.load(f)

user_dataset_lists = [data[0] for key, data in user_dataset.items()]
sample_key = 163225

In [21]:
features_dataframe = pd.DataFrame.from_dict(user_dataset, orient='index',columns=['data', 'activity_label'])
baseline_data = pd.read_csv(path_to_baseline_dataset, dtype={"HEIGHT": float, "AGE":float, "BMI": float, "GENDER": str, "METS_NCEP":float, "HYPERTENSION":float, "DIABETES3":float, "AHI_GE15":float}, low_memory=False)
baseline_dataset_filtered = baseline_data[baseline_data['pid'].isin(user_dataset.keys())]

columns_of_interest = ["pid", "AGE", "GENDERNUM", "HEIGHT", "BMI", "DIABETES3", "HYPERTENSION", "AHI_GE15", "METS_NCEP"]

baselined_dataset_filtered_cols_of_interest = baseline_dataset_filtered[[c for c in baseline_dataset_filtered.columns if c in columns_of_interest]]

sueno_dataset = pd.read_csv(path_to_sueno_dataset, low_memory=False)
sueno_dataset_filtered = sueno_dataset[sueno_dataset['PID'].isin(user_dataset.keys())]

columns_of_interest = ["PID", "ISI_C4"]

sueno_dataset_filtered_cols_of_interest = sueno_dataset_filtered[[c for c in sueno_dataset_filtered.columns if c in columns_of_interest]]
sueno_dataset_filtered_cols_of_interest['ISI_C4'] = sueno_dataset_filtered_cols_of_interest['ISI_C4'].map({1.0:1.0, 2.0:2.0,3.0:3.0, 4.0:3.0})

baseline_sueno_merged = baselined_dataset_filtered_cols_of_interest.set_index('pid').join(sueno_dataset_filtered_cols_of_interest.set_index('PID'))
baseline_sueno_merged["PID"] = baseline_sueno_merged.index
baseline_sueno_merged_no_na = baseline_sueno_merged.dropna()
baseline_sueno_merged_no_na = baseline_sueno_merged_no_na.rename(columns={'DIABETES3':'diabetes',
                                                                                            'METS_NCEP':'metabolic_syndrome',
                                                                                            'ISI_C4':'insomnia',
                                                                                            'HYPERTENSION':'hypertension',
                                                                                            'AHI_GE15':'sleep_apnea'})

In [32]:
df_describe = pd.DataFrame(user_dataset[sample_key][0])
x = df_describe.describe()
x_list = x.values.flatten().tolist()


40

In [23]:
y_labels = np.array(baseline_sueno_features_merged['DIABETES3'].to_list())
x_data = np.array(baseline_sueno_features_merged['data'].to_list())

NameError: name 'baseline_sueno_features_merged' is not defined

In [34]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(x_data, y_labels)

ValueError: setting an array element with a sequence.